In [86]:
import wfdb
import numpy as np
import pygame

pygame.init()
font = pygame.font.Font(None, 36)

def upsample_fractional(signal, factor):
    """
    Upsample a 2D signal by a given factor using linear interpolation.
    Each row is assumed to be a separate channel or variable, and columns represent time.

    :param signal: Input 2D signal as a numpy array (channels x time)
    :param factor: Upsampling factor (can be integer or float)
    :return: Upsampled 2D signal
    """
    # Get the original dimensions
    num_channels, original_time = signal.shape

    # Calculate the new time dimension
    new_time = int(original_time * factor)

    # Create old and new time axes
    old_time_indices = np.arange(original_time)
    new_time_indices = np.linspace(0, original_time - 1, new_time)

    # Initialize the output array
    upsampled_signal = np.zeros((num_channels, new_time))

    # Interpolate each channel
    for channel in range(num_channels):
        upsampled_signal[channel, :] = np.interp(
            new_time_indices, old_time_indices, signal[channel, :])

    return upsampled_signal

def get_color(value):
    value = min(value, 1)
    value = max(value, 0)
    # Example color mapping function
    return int(value * 255), 0, 255 - int(value * 255)

def viz_emg(screen, emg_frame):
    left = 50
    top = 50
    scale = 20
    size = 1
    
    for i in range(4):
        for j in range(8):
            for k in range(8):
                pygame.draw.rect(screen, get_color(emg_frame[i*64+j*k]),
                                 (left+(i//2)*scale*10+scale*j,
                                  top+(i%2)*scale*10+scale*k,
                                  scale*size,
                                  scale*size))
                
def viz_force(screen, force_frame):
    left = 500
    top = 100
    scale = 20
    size = 1

    def viz_force_extensor_helper(screen, x, y, data):
        pygame.draw.rect(screen, get_color(data), (left+scale*x, top+scale*y, scale*size, scale*size))

    viz_force_extensor_helper(screen, 1, 5, force_frame[0])
    viz_force_extensor_helper(screen, 2, 1, force_frame[1])
    viz_force_extensor_helper(screen, 3, 1, force_frame[2])
    viz_force_extensor_helper(screen, 4, 1, force_frame[3])
    viz_force_extensor_helper(screen, 5, 1, force_frame[4])
    

    

In [87]:
# Read a WFDB record
record_force = wfdb.rdrecord('../data/1dof_dataset/subject10_session1/1dof_force_finger4_sample1')
record_emg = wfdb.rdrecord('../data/1dof_dataset/subject10_session1/1dof_preprocess_finger4_sample1')

# Display basic information about the record
print(record_force.__dict__)
print(record_emg.__dict__)

{'record_name': '1dof_force_finger4_sample1', 'n_sig': 5, 'fs': 100, 'counter_freq': None, 'base_counter': None, 'sig_len': 2500, 'base_time': None, 'base_date': None, 'comments': [], 'sig_name': ['thumb', 'index', 'middle', 'ring', 'little'], 'p_signal': array([[-0.00259138, -0.0060374 , -0.00338606,  0.00074284,  0.00176935],
       [-0.00193139, -0.00702769,  0.00417309,  0.00205025,  0.00242968],
       [-0.00127141, -0.01129794, -0.0070062 , -0.00155785,  0.00012921],
       ...,
       [ 0.00957865, -0.0060374 , -0.01292675,  0.00666012, -0.0008601 ],
       [ 0.00957865, -0.00406746, -0.01489673,  0.00732231, -0.00251054],
       [ 0.0112186 , -0.00570785, -0.01390644,  0.00666012, -0.00119066]]), 'd_signal': None, 'e_p_signal': None, 'e_d_signal': None, 'file_name': ['1dof_force_finger4_sample1.dat', '1dof_force_finger4_sample1.dat', '1dof_force_finger4_sample1.dat', '1dof_force_finger4_sample1.dat', '1dof_force_finger4_sample1.dat'], 'fmt': ['16', '16', '16', '16', '16'], 'sam

In [88]:
emg_raw = np.array(record_emg.__dict__['p_signal'].T)
force_raw = np.array(record_force.__dict__['p_signal'].T)

num_frames = emg_raw.shape[-1]
force_sampled = np.array(upsample_fractional(force_raw, emg_raw.shape[-1]/force_raw.shape[-1]))

print("emg.shape:"+str(emg_raw.shape))
print("force.shape:"+str(force_raw.shape))
print("force_sampled.shape:"+str(force_sampled.shape))

emg.shape:(256, 51200)
force.shape:(5, 2500)
force_sampled.shape:(5, 51200)


In [89]:
(np.min(emg_raw, keepdims=True), np.max(emg_raw, keepdims=True))

(array([[-0.62468256]]), array([[0.77078343]]))

In [90]:
(np.min(force_sampled, keepdims=True), np.max(force_sampled, keepdims=True))

(array([[-0.18262633]]), array([[0.09543389]]))

In [91]:

#emg_raw = emg_raw - np.mean(emg_raw, axis=1, keepdims=True) # Normalize to first 10 frames

emg_hdsmg_min = np.min(emg_raw, axis=1, keepdims=True)
emg_hdsmg_max = np.max(emg_raw, axis=1, keepdims=True)
emg_hdsmg_range = emg_hdsmg_max - emg_hdsmg_min
emg_final = (emg_raw - emg_hdsmg_min) / emg_hdsmg_range

force_min = np.min(force_sampled, keepdims=True)
force_max = np.max(force_sampled, keepdims=True)
force_range = force_max - force_min
force_final = (force_sampled - force_min) / force_range

#force_final = force_sampled

In [92]:
print("emg.final:"+str(np.min(emg_final, keepdims=True)) + str(np.max(emg_final, keepdims=True)))
print("force.final:"+str(np.min(force_final, keepdims=True)) + str(np.max(force_final, keepdims=True)))

emg.final:[[0.]][[1.]]
force.final:[[0.]][[1.]]


In [93]:
fps = 100
class EmgKinVisualizer:
    def __init__(self, width=800, height=600):
        self.speed = 30
        self.width = width
        self.height = height
        self.current_frame = 0
        self.running = False

        self.screen = pygame.display.set_mode(
            (width, height + 50))  # Extra height for timeline
        pygame.display.set_caption("EMG and Kinematics Visualizer")

        self.timeline_rect = pygame.Rect(50, height + 10, width - 100, 30)
        self.slider_rect = pygame.Rect(50, height + 10, 10, 30)
        self.exit_button_rect = pygame.Rect(700, 10, 60, 30)

    def run(self):
        clock = pygame.time.Clock()

        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    return
                elif event.type == pygame.KEYDOWN:
                    self.handle_key_event(event.key)
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    if event.button == 1:  # Left mouse button
                        if self.timeline_rect.collidepoint(event.pos):
                            self.update_frame_from_mouse(event.pos[0])
                        if self.exit_button_rect.collidepoint(event.pos):
                            pygame.quit()
                            return
                elif event.type == pygame.MOUSEMOTION:
                    if event.buttons[0]:  # Left mouse button held down
                        if self.timeline_rect.collidepoint(event.pos):
                            self.update_frame_from_mouse(event.pos[0])

            if self.running:
                self.current_frame += self.speed
                if self.current_frame >= num_frames:
                    self.current_frame = 0
                elif self.current_frame < 0:
                    self.current_frame = num_frames - 1

            self.draw()
            pygame.display.flip()
            clock.tick(fps)

    def draw(self):
        self.screen.fill((0, 0, 0))
        
        # Draw emg data
        viz_emg(self.screen, emg_final[:, self.current_frame])

        # Draw glove data
        viz_force(self.screen, force_final[:, self.current_frame])

        # Draw timeline
        pygame.draw.rect(self.screen, (100, 100, 100), self.timeline_rect)
        slider_x = int(self.timeline_rect.left +
                       (self.current_frame / num_frames) * self.timeline_rect.width)
        self.slider_rect.left = slider_x
        pygame.draw.rect(self.screen, (200, 200, 200), self.slider_rect)

        # Draw current frame number
        text = font.render(
            f"Frame: {self.current_frame}/{num_frames-1}", True, (200, 200, 200))
        self.screen.blit(text, (10, self.height - 30))

        # Draw exit button
        pygame.draw.rect(self.screen, (200, 50, 50), self.exit_button_rect)
        exit_text = font.render("Exit", True, (255, 255, 255))
        self.screen.blit(exit_text, (self.exit_button_rect.x + 10, self.exit_button_rect.y + 5))

    def handle_key_event(self, key):
        if key == pygame.K_SPACE:
            self.running = not self.running
        elif key == pygame.K_RIGHT:
            self.speed = 1
            self.running = True
        elif key == pygame.K_LEFT:
            self.speed = -1
            self.running = True
        elif key == pygame.K_UP:
            self.speed = min(self.speed * 2, 16)
        elif key == pygame.K_DOWN:
            self.speed = max(self.speed // 2, 1)

    def update_frame_from_mouse(self, mouse_x):
        timeline_x = mouse_x - self.timeline_rect.left
        frame_ratio = timeline_x / self.timeline_rect.width
        self.current_frame = int(frame_ratio * num_frames)
        self.current_frame = max(0, min(self.current_frame, num_frames - 1))
        self.running = False


if __name__ == "__main__":
    visualizer = EmgKinVisualizer()
    visualizer.run()